# Agent-Based Modeling of Marriages from Online Dating

#### Serena Chen and Apurva Raman

### Replicating the experiment in the [Strength of Absent Ties: Social Integration via Online Dating](https://arxiv.org/pdf/1709.10478.pdf.)

In [5]:
from __future__ import print_function, division

# %matplotlib inline
# %precision 3

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

# from matplotlib import rc
# rc('animation', html='html5')

In [75]:
def getGender(agents_per_race, race):
    women_per_race = agents_per_race//2
    women_matrix = np.ones((women_per_race, race))
    men_matrix= np.zeros((women_per_race, race))
    leftovers = np.random.randint(low=0, high=2, size=(agents_per_race-2*women_per_race,race))# high is exclusive, generates 0 and 1
    gender_matrix = np.concatenate((women_matrix, men_matrix, leftovers))
    
    # generate random matrix to shuffle columns of gender_matrix
    rand_matrix = np.random.random(gender_matrix.shape)
    
    row_indices = np.argsort(rand_matrix, axis=0)
    
    # Preserve order of columns
    col_indices = np.arange(gender_matrix.shape[1])[None, :]
   
    gender_matrix = gender_matrix[row_indices, col_indices]
    
    return np.transpose(gender_matrix)

In [80]:
agents_per_race = 3
races = 2
genders=getGender(agents_per_race,races)
print(genders)

[[ 1.  0.  1.]
 [ 1.  0.  1.]]


In [81]:
def createPersonality(agents_per_race, race):
    social_beliefs = np.random.randint(low=0, high=2, size=(1, race*agents_per_race))
    political_beliefs = np.random.randint(low=0, high=2, size=(1, race*agents_per_race))
    return (social_beliefs, political_beliefs)
   

In [82]:
social_beliefs, political_beliefs = createPersonality(agents_per_race, races)
print ((social_beliefs,political_beliefs))

(array([[0, 0, 0, 0, 1, 1]]), array([[0, 1, 1, 1, 0, 0]]))


In [113]:
def createAdj(agents_per_race, race, prob_inter=0.4, prob_intra=0.7):
    agents = agents_per_race*race
    adj= np.zeros((agents,agents))
    
    #Interracial edges
    max_inter = int(((race-1)*race/2)*agents_per_race**2)
    num_inter = int(max_inter*prob_inter)
    inter_edges = np.random.choice(max_inter, size=(1,num_inter), replace=False)
    inter_edge = 0
    for i in range(agents):
        prev_race = i//agents_per_race
        for j in range(prev_race*agents_per_race+agents_per_race, agents):
            if (i != j):
                if np.any(inter_edges == inter_edge):
                    adj[i,j] = 1
                inter_edge +=1    
                
    #Intraracial edges
    intra_edges = np.random.rand(agents,agents)
    for i in range(agents):
        current_race = i//agents_per_race
        for j in range (i+1, current_race*agents_per_race+agents_per_race):           
            if (intra_edges[i,j] < prob_intra):
                adj[i,j] = 1
                
    #Reflect triangle
    adj = np.triu(adj,0)+np.transpose(np.triu(adj))
    
    #Get all connections of at least distance 2 (at least one mutual friend)
    adj2 = np.matmul(adj,adj)
    
    #Directly connected or have at least one mutual friend
    adj3 = (adj+adj2>0).astype(int)
    np.fill_diagonal(adj3, 0)
    
    return adj,adj2,adj3

In [114]:
(createAdj(agents_per_race, races))

(array([[ 0.,  0.,  0.,  1.,  0.,  1.],
        [ 0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  0.],
        [ 1.,  0.,  1.,  0.,  1.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  1.],
        [ 1.,  0.,  0.,  1.,  1.,  0.]]),
 array([[ 2.,  0.,  1.,  1.,  2.,  1.],
        [ 0.,  1.,  0.,  1.,  0.,  0.],
        [ 1.,  0.,  2.,  0.,  1.,  1.],
        [ 1.,  1.,  0.,  4.,  1.,  2.],
        [ 2.,  0.,  1.,  1.,  2.,  1.],
        [ 1.,  0.,  1.,  2.,  1.,  3.]]),
 array([[0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 0, 0],
        [1, 1, 0, 1, 1, 1],
        [1, 1, 1, 0, 1, 1],
        [1, 0, 1, 1, 0, 1],
        [1, 0, 1, 1, 1, 0]]))